# Semantic Kernel with Azure OpenAI ⚡

This notebook is a simple demonstration of how to use the [Semantic Kernel](https://learn.microsoft.com/semantic-kernel/overview/?WT.mc_id=AZ-MVP-5003203) with [Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service?WT.mc_id=AZ-MVP-5003203). 

## Install requirements
Make sure you have configured the necessary settings for the Azure OpenAI Connector. See the [documentation](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md#openai--azure-openai-api-keys).

We only need to install the Semantic Kernel SDK from pypi.org

In [ ]:
%pip install semantic-kernel

## Initialize the Kernel

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()
service_id="chat-gpt"

kernel.add_service(
  AzureChatCompletion(
      service_id=service_id,
  )
)

## Add Plugins
The following script will add all plugins from the `plugins_directory` to the kernel:

In [ ]:
plugins_directory = "Plugins"
kubernetesPlugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="KubernetesPlugin")

## Run the KubectlPrompt Plugin 

In [ ]:
result = await kernel.invoke(kubernetesPlugin["KubectlPrompt"], input="list all namespace")
print(result)

## Run the KubeMatch Plugin

In [ ]:
goodFit="""
This project involves developing a high-traffic e-commerce platform that expects dynamic scaling and high availability to handle seasonal peaks in user traffic. 
The architecture is based on microservices, each responsible for different aspects such as user authentication, product catalog, and payment processing. 
The team is experienced in containerized environments and requires a solution that allows for efficient deployment, scaling, and management of services, with robust orchestration capabilities.
"""

result = await kernel.invoke(kubernetesPlugin["KubeMatch"], input=goodFit)
print(result)

In [ ]:

badFit="""
This project is a small-scale internal tool for automating office inventory management, expected to have a stable and predictable load with minimal scalability requirements. 
The application is built with a simple, monolithic architecture and will be deployed on a few servers. 
The team lacks experience with containerization and Kubernetes, and the simplicity of the project does not justify the overhead and complexity of implementing a Kubernetes environment.
"""

result = await kernel.invoke(kubernetesPlugin["KubeMatch"], input=badFit)
print(result)

## Add native code as a plugin
The plugin described below allows us to **execute kubectl** commands.

In [ ]:
import subprocess

from semantic_kernel.functions import kernel_function

class ExecuteKubectlPlugin:
    """
    Description: This plugin is used to execute kubectl commands
    """

    @kernel_function(
        description="Execute kubectl command",
        name="ExecuteKubectl",
    )
    def execute_kubectl_command (self, input: str) -> str:
        """
        Execute kubectl command
        """
        try:
            command = input.replace("kubectl", "").strip() # remove kubectl from the input
            result = subprocess.run(['kubectl'] + command.split(), capture_output=True, text=True, check=True)

            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing kubectl command: {e}")


### Add the native plugin to the kernel

In [ ]:
execute_kubectl_plugin = kernel.add_plugin(plugin=ExecuteKubectlPlugin(), plugin_name="ExecuteKubectlPlugin")

In [ ]:
execute_kubectl_fnc = execute_kubectl_plugin["ExecuteKubectl"]
kubectl_output = await execute_kubectl_fnc(kernel,input="kubectl cluster-info")
print(kubectl_output)


## Using the Planner 🪄

In [ ]:
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)

question = "list all namepsaces"

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000,
)

planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)
result = await planner.invoke(kernel, question)

### Print the plan

In [ ]:
print(f"Chat history: {result.chat_history}\n")